<a href="https://colab.research.google.com/github/jedsadakorn19/NLP_code/blob/main/label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pythainlp
!pip install "tensorflow>=1.9,<2.0"
!pip install --upgrade tensorflow-hub
!pip install bert-for-tf2
!pip install sklearn_crfsuite
!pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
!pip install fastai==1.0.46
!pip install emoji
!pip install transformers
!pip install "sentencepiece==0.0.9"
!pip3 install tf-sentencepiece
# !pip install tensorflow-hub

In [ ]:
import pythainlp
pythainlp.__version__

In [ ]:
# from google.colab import files
# uploaded = files.upload()
!git clone https://github.com/thirasan/bert-1.git

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import codecs
import re,urllib
import numpy as np
from pythainlp.tokenize import word_tokenize,Tokenizer
from nltk.tokenize import RegexpTokenizer
import tensorflow as tf
import tensorflow_hub as hub
from bert import bert_tokenization
from pythainlp.tokenize import word_tokenize
from pythainlp.tokenize import Tokenizer
from pythainlp.util.trie import Trie, dict_trie
from pythainlp.corpus.common import thai_words
from pythainlp.corpus import thai_stopwords
from tqdm import tqdm_notebook
from tensorflow import keras
from tensorflow.keras import layers
from IPython.display import Image 
import datetime
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score, confusion_matrix, f1_score
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import dill as pickle
from transformers import BertTokenizer
import pandas as pd
import torch
from pythainlp import word_vector
from transformers import AutoTokenizer, AutoModel
model_path = 'thwiki_data/models/'

In [ ]:
model = AutoModel.from_pretrained("monsoon-nlp/bert-base-thai")
# model = word_vector.get_model()

In [ ]:
file_name="1702" # ชื่อไฟล์คลังข้อมูล
def get_data(fileopen):
	"""
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
	with codecs.open(fileopen, 'r',encoding='utf-8-sig') as f:
		lines = f.read().splitlines()
	return [a for a in lines if Unique(a)]# เอาไม่ซ้ำกัน

In [ ]:
data_not=[]
def Unique(p):
 text=re.sub("<[^>]*>","",p)
 text=re.sub("\[(.*?)\]","",text)
 text=re.sub("\[\/(.*?)\]","",text)   
 if text not in data_not:
  data_not.append(text)
  return True
 else:
  return False

In [ ]:
rawdata = get_data("1702.txt")

In [ ]:
print(len(rawdata))

In [ ]:
def toolner_to_tag(text):
 text=text.strip()
 text=re.sub("<[^>]*>","",text)
 text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')#  ตัดการกับพวกไม่มี tag word
 text=re.sub("(\[\w+\])","***\\1",text)
 text2=[]
 for i in text.split('***'):
  if "[" in i:
   text2.append(i)
  # else:
  #  text2.append("[word]"+i+"[/word]")
 text="".join(text2)#re.sub("[word][/word]","","".join(text2))
 return text.replace("[word][/word]","")

In [ ]:
def text2conll2002(text,pos=True):
    text=toolner_to_tag(text) # นำไปใส่ tag [word] "[word]เราจะไปเดินเล่นที่[/word]"
    text=text.replace("''",'"')
    text=text.replace("(",'')
    text=text.replace(")",'')
    text=text.replace("[Pronouns]",'')
    text=text.replace("[noun]",'')
    text=text.replace("[intransitive verb]",'')
    text=text.replace("[transitive verb]",'')
    text=text.replace("[Denial word]",'')
    text=text.replace("[word before verb]",'')
    text=text.replace("[word after verb]",'')
    text=text.replace("[Adverbs]",'')
    text=text.replace("[Adverb]",'')
    text=text.replace("[Qualitative noun]",'')
    text=text.replace("[Adjectives]",'')
    text=text.replace("[Numeral word]",'')
    text=text.replace("[Sequence word]",'')
    text=text.replace("[word before numeral]",'')
    text=text.replace("[word after numeral]",'')
    text=text.replace("[Time expression word]",'')
    text=text.replace("[Conclusion word]",'')
    text=text.replace("[Special word]",'')
    text=text.replace("[Prepositions]",'')
    text=text.replace("[Conjunctions]",'')
    text=text.replace("[SS Conjunctions]",'')
    text=text.replace("[Interjections]",'')
    text=text.replace("[abbreviation]",'')
    text=text.replace("[word before noun]",'')
    text=text.replace("[Phrasal Verbs]",'')
    text=text.replace("[Noun phrase]",'')
    text=text.replace("[Noun Conjunctions]",'')
    text=text.replace("[Negator]",'')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer1.tokenize(text) # แยก tag ออกมาจากข้อความ [('word', 'ฉันจะไปเดินเล่นที่', 'word'),('LOCATION', 'หนองคาย', 'LOCATION'),('word', 'พร้อมกับนั่งเรือข้ามไป', 'word'),('LOCATION', 'ประเทศลาว', 'LOCATION')]
    return tag

In [ ]:
pattern = '(.*?)\[\/(.*?)\]'
tokenizer1 = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น ('TIME','ไง','TIME')

In [ ]:
text2conll2002(rawdata[0])

In [ ]:
print((rawdata[65]))

In [ ]:
d=[]
o=[]
for i in range(len(rawdata[0:75])):
    r=text2conll2002(rawdata[i])
    d.append(r)
for i in range(76,90):
    m=text2conll2002(rawdata[i])
    o.append(m)    

In [ ]:
o

In [ ]:
d

In [ ]:
train_sentences = d
test_sentences  = o

In [ ]:
print(len(train_sentences))
print("Tagged words in train set:", len([item for sublist in train_sentences for item in sublist]))
print(len(test_sentences))
print("Tagged words in test set:", len([item for sublist in test_sentences for item in sublist]))

In [ ]:
def tag_sequence(sentences):
    return [[t for w, t in sentence] for sentence in sentences]

def text_sequence(sentences):
    return [[w for w, t in sentence] for sentence in sentences]

In [ ]:
tags1 = set([item for sublist in test_sentences for _, item in sublist])
tags2 = set([item for sublist in train_sentences for _, item in sublist])
print('TOTAL TAGS: ', len(tags1))
print('TOTAL TAGS: ', len(tags2))

In [ ]:
tags = set([item for sublist in train_sentences+test_sentences for _, item in sublist])
print('TOTAL TAGS: ', len(tags))

In [ ]:
print(tags1)
print(tags2)

In [ ]:
print(tags)

In [ ]:
tag2int = {}
int2tag = {}

for i, tag in enumerate(sorted(tags)):
    tag2int[tag] = i+1
    int2tag[i+1] = tag

In [ ]:
tag2int['-PAD-'] = 0
int2tag[0] = '-PAD-'

In [ ]:
n_tags = len(tag2int)
print('Total tags:', n_tags)

In [ ]:
print(tag2int)

In [ ]:
MAX_SEQUENCE_LENGTH = 70
EPOCHS = 20

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.hist([len(s) for s in train_sentences], bins=50)
plt.show()

In [ ]:
print('Max sentence length:',len(max(rawdata, key=len)))

In [ ]:
def  split(sentences, max):
    new=[]
    for data in sentences:
        new.append(([data[x:x+max] for x in range(0, len(data), max)]))
    new = [val for sublist in new for val in sublist]
    return new

In [ ]:
train_sentences = split(train_sentences, MAX_SEQUENCE_LENGTH)
test_sentences  = split(test_sentences, MAX_SEQUENCE_LENGTH)

In [ ]:
test_sentences

In [ ]:
train_sentences

In [ ]:
len(max(train_sentences, key=len))

In [ ]:
# Initialize session
sess = tf.compat.v1.Session()
# Params for bert model and tokenization
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

In [ ]:
train_text = text_sequence(train_sentences)
train_label = tag_sequence(train_sentences)
test_text = text_sequence(test_sentences)
test_label= tag_sequence(test_sentences)

In [ ]:
len(test_label)

In [ ]:
train_text[0], train_label[0]

In [ ]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    # bert_module =  hub.load(bert_path)
    # tokenization_info = bert_module.signatures['tokenization_info']
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return  bert_tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

def convert_single_example(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        label_ids = [0] * max_seq_length
        return input_ids, input_mask, segment_ids, label_ids
    
    tokens_a = example.text_a
    if len(tokens_a) > max_seq_length-2:
        tokens_a = tokens_a[0 : (max_seq_length-2)]

# Token map will be an int -> int mapping between the `orig_tokens` index and
# the `bert_tokens` index.

# bert_tokens == ["[CLS]", "john", "johan", "##son", "'", "s", "house", "[SEP]"]
# orig_to_tok_map == [1, 2, 4, 6]   
    orig_to_tok_map = []              
    tokens = []
    segment_ids = []
    
    tokens.append("[CLS]")
    segment_ids.append(0)
    orig_to_tok_map.append(len(tokens)-1)
    #print(len(tokens_a))
    for token in tokens_a:       
        tokens.extend(tokenizer.tokenize(token))
        #tokens.extend(_tokenizer.word_tokenize(token))
        
        orig_to_tok_map.append(len(tokens)-1)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)
    orig_to_tok_map.append(len(tokens)-1)
    input_ids = tokenizer.convert_tokens_to_ids([tokens[i] for i in orig_to_tok_map])
    #print(len(orig_to_tok_map), len(tokens), len(input_ids), len(segment_ids)) #for debugging

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)
    
    label_ids = []
    labels = example.label
    label_ids.append(0)
    label_ids.extend([tag2int[label] for label in labels])
    label_ids.append(0)
    #print(len(label_ids)) #for debugging
    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)
        label_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    assert len(label_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, label_ids

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels),
    )

def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=text, text_b=None, label=label)
        )
    return InputExamples

In [ ]:
# tokenizer = create_tokenizer_from_hub_module()

In [ ]:
cd ..

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('monsoon-nlp/bert-base-thai')
from bert1.tokenization import *
tokenizer = ThaiTokenizer(vocab_file='m.vocab', spm_file='m.model')  

In [ ]:
custom_words_list = set(thai_words())
custom_words_list.add('การที่รองอัยการสูงสุด')
custom_words_list.add('ผบ.ตร.')
custom_words_list.add('คำสั่งฟ้อง')
custom_words_list.add('ก.ค.')
trie = dict_trie(dict_source=custom_words_list)
_tokenizer = Tokenizer(custom_dict=trie, engine='newmm')

In [ ]:
def bert_labels(labels):
    train_label_bert = []
    train_label_bert.append('-PAD-')
    for i in labels:
        train_label_bert.append(i)
    train_label_bert.append('-PAD-')
    print('BERT labels:', train_label_bert)

In [ ]:
tokens_a = train_text[2]
tokens_a

In [ ]:

orig_to_tok_map = []              
tokens = []
segment_ids = []
tokens.append("[CLS]")
segment_ids.append(0)
orig_to_tok_map.append(len(tokens)-1)
for token in tokens_a:
    #orig_to_tok_map.append(len(tokens)) # keep first piece of tokenized term
    tokens.extend(tokenizer.tokenize(token))
    #tokens.extend(_tokenizer.word_tokenize(token))
    orig_to_tok_map.append(len(tokens)-1) # # keep last piece of tokenized term -->> gives better results!
    segment_ids.append(0)    
tokens.append("[SEP]")
print(tokens)
segment_ids.append(0)
print(segment_ids)
orig_to_tok_map.append(len(tokens)-1)
print(orig_to_tok_map)
input_ids = tokenizer.convert_tokens_to_ids([tokens[i] for i in orig_to_tok_map])
print(input_ids)
print(len(input_ids))

In [ ]:
print('Original tokens:',tokens_a)
print(len(tokens_a))

In [ ]:
print('Original tokens:',tokens_a[2])
print('Original tokens:',train_text[2][2])

In [ ]:
print(train_text[2])
print(len(train_text[2]))

In [ ]:
print("Labal:",train_label[2])
print(len(train_label[2]))

In [ ]:
print('BERT tokens:',tokens)
print(len(tokens))

In [ ]:
bert_labels(train_label[2])
print(len(train_label[2]))

In [ ]:
print(len(tokens))
print(len(train_label[2]))

In [ ]:
orig_to_tok_map

In [ ]:
print(input_ids)
print(len(input_ids))

In [ ]:
"""Create InputExamples"""
InputExamples = []
for text, label in zip(train_text[0:1], train_label[0:1]):
    InputExamples.append(
        InputExample(guid=None, text_a=text, text_b=None, label=label)
)

In [ ]:
input_ids, input_masks, segment_ids, labels = [], [], [], []
for example in tqdm_notebook(InputExamples, desc="Converting examples to features"):
    input_id, input_mask, segment_id, label = convert_single_example(
        tokenizer, example, MAX_SEQUENCE_LENGTH+2
    )
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)
    labels.append(label)
    print(input_id)
    print(input_ids)

In [ ]:
print(input_id)
print(len(input_id))

In [ ]:
print(input_ids)
print(len(input_ids))

In [ ]:
test_text

In [ ]:
test_label

In [ ]:
train_examples = convert_text_to_examples(train_text, train_label)
test_examples = convert_text_to_examples(test_text, test_label)

In [ ]:
train_label[2]

In [ ]:
# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels_ids
) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=MAX_SEQUENCE_LENGTH+2)
(test_input_ids, test_input_masks, test_segment_ids, test_labels_ids
) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=MAX_SEQUENCE_LENGTH+2)

In [ ]:
print(len(train_input_ids[0]))
print(len(train_input_masks[0]))
print(len(train_segment_ids[0]))
print(len(train_labels_ids[0]))

In [ ]:
train_input_ids[1]

In [ ]:
train_input_masks[0]

In [ ]:
train_segment_ids[0]

In [ ]:
train_labels_ids[0]

In [ ]:
# One-hot encode labels
train_labels = keras.utils.to_categorical(train_labels_ids, num_classes=n_tags)
test_labels = keras.utils.to_categorical(test_labels_ids, num_classes=n_tags)

In [ ]:
train_input_ids[0], train_labels[0]

In [ ]:
# bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
bert_path = "https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1"

In [ ]:
class BertLayer(keras.layers.Layer):
    def __init__(self, output_representation='sequence_output', trainable=True, **kwargs):
        self.bert = None
        super(BertLayer, self).__init__(**kwargs)
        
        self.trainable = trainable
        self.output_representation = output_representation

    def build(self, input_shape):
        # SetUp tensorflow Hub module
        self.bert = hub.Module(bert_path,
                               trainable=self.trainable, 
                               name="{}_module".format(self.name))

        # Assign module's trainable weights to model
        # Remove unused layers and set trainable parameters
        # s = ["/cls/", "/pooler/", 'layer_11', 'layer_10', 'layer_9', 'layer_8', 'layer_7', 'layer_6']
        s = ["/cls/", "/pooler/"]
        self._trainable_weights += [var for var in self.bert.variables[:] if not any(x in var.name for x in s)]
            
        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)
                
        # See Trainable Variables
        #tf.logging.info("**** Trainable Variables ****")
        #for var in self.trainable_weights:
        #    init_string = ", *INIT_FROM_CKPT*"
        #    tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape, init_string)
            
        print('Trainable weights:',len(self._trainable_weights))
        super(BertLayer, self).build(input_shape)

    def call(self, inputs, mask=None):
        inputs = [keras.backend.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
            self.output_representation
        ]
        return result

    def compute_mask(self, inputs, mask=None):
        return keras.backend.not_equal(inputs[0], 0.0)   

    def compute_output_shape(self, input_shape):
        if self.output_representation == 'pooled_output':
            return (None, 768)
        else:
            return (None, None, 768)

In [ ]:
# Build model
def build_model(max_seq_length):
    seed = 0 
    in_id = keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    np.random.seed(seed)
    bert_output = BertLayer()(bert_inputs)

    np.random.seed(seed)
    outputs = keras.layers.Dense(n_tags, activation=keras.activations.softmax)(bert_output)

    np.random.seed(seed)
    model = keras.models.Model(inputs=bert_inputs, outputs=outputs)
    np.random.seed(seed)
    model.compile(optimizer=keras.optimizers.Adam(lr=0.00004), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])   
    model.summary(100)
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    keras.backend.set_session(sess)

In [ ]:
# import tf_sentencepiece
model = build_model(MAX_SEQUENCE_LENGTH+2) # We sum 2 for [CLS], [SEP] tokens

In [ ]:
keras.utils.plot_model(model, to_file='model.png', show_shapes=True)
Image('model.png')


In [ ]:
train_input_ids.shape

In [ ]:
train_input_masks.shape

In [ ]:
train_segment_ids.shape

In [ ]:
train_labels.shape

In [ ]:
initialize_vars(sess)

In [ ]:
t_ini = datetime.datetime.now()
print(EPOCHS)
EPOCHS=30
cp = keras.callbacks.ModelCheckpoint(filepath="model.ckpt",
                     monitor='val_acc',
                     save_best_only=True,
                     save_weights_only=True,
                     verbose=1)

early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5)

history = model.fit([train_input_ids, train_input_masks, train_segment_ids], 
                    train_labels,
                    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
                    #validation_split=0.3,
                    epochs=EPOCHS,
                    batch_size=16,
                    shuffle=True,
                    verbose=1,
                    callbacks=[cp, early_stopping]
                   ) 

t_fin = datetime.datetime.now()
print('Training completed in {} seconds'.format((t_fin - t_ini).total_seconds()))

In [ ]:
model = build_model(MAX_SEQUENCE_LENGTH+2) 
model.load_weights('model.ckpt')

In [ ]:
y_pred = model.predict([test_input_ids, test_input_masks, test_segment_ids]).argmax(-1)
y_true = test_labels.argmax(-1)

In [ ]:
y_true[0]

In [ ]:
def y2label(zipped, mask=0):
    out_true = []
    out_pred = []
    for zip_i in zipped:
        a, b = tuple(zip_i)
        if a != mask :
            out_true.append(int2tag[a])
            out_pred.append(int2tag[b])
    return out_true, out_pred

In [ ]:
y_zipped = zip(y_true.flat, y_pred.flat)
y_true, y_pred = y2label(y_zipped)

In [ ]:
len(y_true), len(y_pred)

In [ ]:
name='Bert fine-tuned model'
print('\n------------ Result of {} ----------\n'.format(name))
print(classification_report(y_true, y_pred, digits=4))

In [ ]:
print("Accuracy: {0:.4f}".format(accuracy_score(y_true, y_pred)))
print('f1-macro score: {0:.4f}'.format(f1_score(y_true, y_pred, average='macro')))

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Accuracy")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='accuracy')
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
def plot_confusion_matrix(f1,
                          cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True,
                          i=1):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    plt.figure(figsize=(20, 12))
    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}; f1-score={:0.4f}'.format(accuracy, misclass, f1))

In [ ]:
def plot_acc():
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
def plot_loss():
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
tags = sorted(set(y_pred+y_true))
cnf_matrix = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(f1_score(y_true, y_pred, average='macro'), cnf_matrix, target_names=tags, title=name, normalize=False)

In [ ]:
y_pred = model.predict([test_input_ids, test_input_masks, test_segment_ids], batch_size=16).argmax(-1)
y_true = test_labels_ids

def make_prediction(i=16):
    note = ''
    sent = []
    print("{:10} {:5} : {:5}".format("Word", "True", "Predicted"))
    print(35*'-')
    for w, true, pred in zip(test_input_ids[i], y_true[i], y_pred[i]):
        if tokenizer.convert_ids_to_tokens([w])[0]!='[PAD]' and \
            tokenizer.convert_ids_to_tokens([w])[0]!='[CLS]' and \
            tokenizer.convert_ids_to_tokens([w])[0]!='[SEP]':
            if int2tag[true] != int2tag[pred]: note='<<--- Error!'
            print("{:10} {:5} : {:5} {:5}".format(tokenizer.convert_ids_to_tokens([w])[0], int2tag[true], int2tag[pred], note))
            note=''

In [ ]:
make_prediction(i=12)

In [ ]:
sentence_raw1 = 'พี่สาวผงะ พบโครงกระดูกติดซากรถถูกไฟคลอกดับ เชื่อเป็นน้องชายที่หายไปนั้น ได้รับรายงานจากการ'

In [ ]:
sentence_raw=sentence_raw1.replace(" ", "")

In [ ]:
# from pythainlp.tokenize import word_tokenize
sentence_ini = word_tokenize(sentence_raw)

In [ ]:
sentence_ini

In [ ]:
tokens_a = sentence_ini

In [ ]:
orig_to_tok_map = []              
tokens = []
segment_ids = []
tokens.append("[CLS]")
segment_ids.append(0)
orig_to_tok_map.append(len(tokens)-1)
for token in tokens_a:
    #orig_to_tok_map.append(len(tokens)) # keep first piece of tokenized term
    tokens.extend(tokenizer.tokenize(token))
    # tokens.extend(_tokenizer.word_tokenize(token))
    orig_to_tok_map.append(len(tokens)-1) # # keep last piece of tokenized term -->> gives better results!
    segment_ids.append(0)
tokens.append("[SEP]")
segment_ids.append(0)
orig_to_tok_map.append(len(tokens)-1)
input_ids = tokenizer.convert_tokens_to_ids([tokens[i] for i in orig_to_tok_map])

In [ ]:
# print('Original tokens:',tokens_a)

In [ ]:
print('BERT tokens:',tokens)

In [ ]:
orig_to_tok_map

In [ ]:
[tokens[i] for i in orig_to_tok_map]

In [ ]:
test_example = convert_text_to_examples([sentence_ini], [['-PAD-']*len(sentence_ini)])

In [ ]:
# Convert to features
(input_ids, input_masks, segment_ids, train_labels_ids
) = convert_examples_to_features(tokenizer, test_example, max_seq_length=MAX_SEQUENCE_LENGTH+2)

In [ ]:
predictions = model.predict([input_ids, input_masks, segment_ids], batch_size=1).argmax(-1)[0]
print("\n{:20}| {:15}: {:15}".format("Word in BERT layer", 'Initial word', "Predicted POS-tag"))
print(61*'-')
k = 0
for i, pred in enumerate(predictions):
    try:
        if pred!=0:
            print("{:20}| {:15}: {:15}".format([tokens[i] for i in orig_to_tok_map][i], sentence_ini[i-1], int2tag[pred]))            
            k+=1            
    except:
        pass

In [ ]:
def find_errors(X,y):
    error_counter = collections.Counter()
    support = 0
    for i in range(test_input_ids.shape[0]):
        for w, true, pred in zip(test_input_ids[i], y_true[i], y_pred[i]):
            if int2tag[true]!='-PAD-':
                if true != pred:
                    word = tokenizer.convert_ids_to_tokens([w])[0]
                    error_counter[word] += 1
                support += 1
    return error_counter, support

In [ ]:
errors, support = find_errors([test_input_ids],y_true)

In [ ]:
print('Total errors/Total words: {}/{} | Accuracy: {:.4}\n'.format(sum(errors.values()), support, 1-sum(errors.values())/support))
print('Most common errors:', errors.most_common(20))

In [ ]:
plot_acc()

In [ ]:
plot_loss()